In [ ]:
from dataclasses import dataclass
from transformer_lens import HookedTransformer
import core

%load_ext autoreload
%autoreload 2

In [ ]:
# Models
gpt2 = HookedTransformer.from_pretrained("gpt2-small")
attn_only_4l = HookedTransformer.from_pretrained("attn-only-4l")

In [ ]:
@dataclass
class Task:
    model: HookedTransformer
    prompt: str
    corrupted: list[str]
    correct: str
    incorrect: list[str]
    short_name: str

In [ ]:
task_ioi_ABBA = Task(
    model=gpt2,
    prompt="When Mary and John went to the store, John gave a book to",
    correct=" Mary",
    incorrect=[" John"],
    corrupted=["When Felix and Sarah went to the store, Diego gave a book to",
               "When Felix and Sarah went to the store, Felix gave a book to"],
    short_name="ioi_abba",
)

task_ioi_BABA = Task(
    model=gpt2,
    prompt="When John and Mary went to the store, John gave a book to",
    correct=" Mary",
    incorrect=[" John"],
    corrupted=["When Felix and Sarah went to the store, Diego gave a book to",
               "When Felix and Sarah went to the store, Felix gave a book to"],
    short_name="ioi_baba",
)
task_docstring = Task(
    model=attn_only_4l,
    prompt = '''def port(self, load, size, file, last):
    """oil column piece

    :param load: crime population
    :param size: unit dark
    :param''',
    correct = " file",
    incorrect = [" last", " load", " size", " self"],
    corrupted = [],  # TODO: fill in
    short_name="docstring 1",
)
task_docstring_2 = Task(
    model=attn_only_4l,
    prompt = '''def run(place, nb, last, first):
    """run the place

    :param place: place to run
    :param nb: number of times to run
    :param''',
    correct = " first",
    incorrect = [" last", " place", " nb"],
    corrupted = [],  # TODO: fill in
    short_name="docstring 2",
)
TASKS = [task_ioi_ABBA, task_ioi_BABA]

In [ ]:
threshold = 0.3
top_k = 10
for task in TASKS:
    metric = core.logit_diff_metric(task.model, task.correct, *task.incorrect)
    assert metric(task.model(task.prompt)[0]) > 0.1, "The model cannot do the task"
    interventions = [
        core.ZeroPattern(),
        core.DampenIntervention(0.1),
        core.DampenIntervention(0.3),
        core.CropIntervention(task.model, task.prompt),
    ]
    # for corrupted in task.corrupted:
    #     interventions.append(core.CorruptIntervention(task.model, task.prompt, corrupted))

    strategy = core.BasicStrategy()
    for intervention in interventions:
        connectome = core.connectom(task.model, task.prompt, metric, intervention, strategy)
        print(task.short_name)
        print(task.prompt)
        print("Correct: ", task.correct)
        print("Incorrect: ", task.incorrect)
        print("Intervention: ", intervention)
        core.plot_graphviz_connectome(task.model, task.prompt, connectome, top_k=top_k)
